# Selecting and Tuning pipelines

This guide shows you how to search for multiple pipelines for your problem
and later on use a [BTBSession](https://hdi-project.github.io/BTB/api/btb.session.html#btb.session.BTBSession)
to select and tune the best one.

Note that some steps are not explained for simplicity. Full details
about them can be found in the previous parts of the tutorial.

Here we will:

1. Load a dataset
2. Search and load suitable templates
3. Write a scoring function
4. Build a BTBSession for our templates
5. Run the session to find the best pipeline

## Load the Dataset

The first step will be to load the dataset.

In [1]:
from mlprimitives.datasets import load_dataset

dataset = load_dataset('census')

In [2]:
dataset.describe()

Adult Census dataset.

    Predict whether income exceeds $50K/yr based on census data. Also known as "Adult" dataset.

    Extraction was done by Barry Becker from the 1994 Census database. A set of reasonably clean
    records was extracted using the following conditions: ((AAGE>16) && (AGI>100) &&
    (AFNLWGT>1)&& (HRSWK>0))

    Prediction task is to determine whether a person makes over 50K a year.

    source: "UCI
    sourceURI: "https://archive.ics.uci.edu/ml/datasets/census+income"
    
Data Modality: single_table
Task Type: classification
Task Subtype: binary
Data shape: (32561, 14)
Target shape: (32561,)
Metric: accuracy_score
Extras: 


## Find and load suitable Templates

We will be using the `mlblocks.discovery.find_pipelines` function to search
for compatible pipelines.

In this case, we will be looking for `single_table/classification` pipelines.

In [3]:
from mlblocks.discovery import find_pipelines

filters = {
    'metadata.data_type': 'single_table',
    'metadata.task_type': 'classification'
}
find_pipelines(filters=filters)

['keras.Sequential.MLPBinaryClassifier',
 'keras.Sequential.MLPMultiClassClassifier',
 'single_table.classification',
 'single_table.classification.text',
 'single_table.classification.xgb',
 'sklearn.decomposition.DictionaryLearning',
 'sklearn.decomposition.FactorAnalysis',
 'sklearn.decomposition.FastICA',
 'sklearn.decomposition.KernelPCA',
 'sklearn.decomposition.PCA',
 'sklearn.decomposition.TruncatedSVD',
 'sklearn.ensemble.AdaBoostClassifier',
 'sklearn.ensemble.BaggingClassifier',
 'sklearn.ensemble.ExtraTreesClassifier',
 'sklearn.ensemble.GradientBoostingClassifier',
 'sklearn.ensemble.IsolationForest',
 'sklearn.ensemble.RandomForestClassifier',
 'sklearn.ensemble.RandomTreesEmbedding',
 'sklearn.linear_model.LogisticRegression']

And we will create a dictionary with MLPipeline instances that will be used as tempaltes for our tuning.

In [4]:
from mlblocks import MLPipeline

templates = [
    'single_table.classification',
    'single_table.classification.text',
    'single_table.classification.xgb',
    'sklearn.ensemble.AdaBoostClassifier',
    'sklearn.ensemble.BaggingClassifier',
    'sklearn.ensemble.ExtraTreesClassifier',
    'sklearn.ensemble.GradientBoostingClassifier',
]

# limit number of estimators in XGB primitives to avoid long training times
init_params = {
    'xgboost.XGBClassifier#1': {
        'n_estimators': 100
    }
}

templates_dict = {
    template: MLPipeline(template, init_params=init_params)
    for template in templates
}

In [5]:
templates_dict['single_table.classification']

## Create a scoring function

In order to use a `BTBSession` we will need a function that is able to score a proposal,
which will always be a pair of template name and proposed hyperparameters.

In this case, the evaluation will be done using 5-fold cross validation over our dataset.

In [6]:
import numpy as np

def cross_validate(template_name, hyperparameters=None):
    template = templates_dict[template_name]
    scores = []
    for X_train, X_test, y_train, y_test in dataset.get_splits(5):
        pipeline = MLPipeline(template.to_dict())  # Make a copy of the template
        if hyperparameters:
            pipeline.set_hyperparameters(hyperparameters)

        pipeline.fit(X_train, y_train)
        y_pred = pipeline.predict(X_test)
        
        scores.append(dataset.score(y_test, y_pred))
        
    return np.mean(scores)

## Setup the BTBSession

We will create another dictionary with the tunable hyperparameters of each template.
This will be used by the BTBSession to know how to tune each template.

In [7]:
tunables = {
    name: template.get_tunable_hyperparameters(flat=True)
    for name, template in templates_dict.items()
}

In [8]:
tunables['single_table.classification']

{('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
  'max_labels'): {'type': 'int', 'default': 0, 'range': [0, 100]},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'lowercase'): {'type': 'bool', 'default': True},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'binary'): {'type': 'bool', 'default': True},
 ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
  'max_features'): {'type': 'int', 'default': 1000, 'range': [1, 10000]},
 ('sklearn.impute.SimpleImputer#1', 'strategy'): {'type': 'str',
  'default': 'mean',
  'values': ['mean', 'median', 'most_frequent', 'constant']},
 ('xgboost.XGBClassifier#1', 'max_depth'): {'type': 'int',
  'default': 3,
  'range': [3, 10]},
 ('xgboost.XGBClassifier#1', 'learning_rate'): {'type': 'float',
  'default': 0.1,
  'range': [0, 1]},
 ('xgboost.XGBClassifier#1', 'gamma'): {'type': 'float',
  'default': 0,
  'range': [0, 1]},
 ('xgboost.XGBClassifier#1', 'min_child_weight'): {'type': 'int',


And then create a `BTBSession` instance passing them and the `cross_validate` function.

We will also be setting it in `verbose` mode, so we can have a better insight on what is going on.

In [9]:
from btb.session import BTBSession

session = BTBSession(tunables, cross_validate, verbose=True)

## 5. Run the session

After everything is set up, we can start running the tuning session passing it
the number of iterations that we want to perform.

In [10]:
session.run(5)

2020-06-09 18:27:33,705 - INFO - session - Creating Tunable instance from dict.
2020-06-09 18:27:33,706 - INFO - session - Obtaining default configuration for single_table.classification
2020-06-09 18:27:39,855 - INFO - session - New optimal found: single_table.classification - 0.8639171383183359
2020-06-09 18:27:39,860 - INFO - session - Creating Tunable instance from dict.
2020-06-09 18:27:39,860 - INFO - session - Obtaining default configuration for single_table.classification.text
2020-06-09 18:27:40,015 - ERROR - mlpipeline - Exception caught producing MLBlock mlprimitives.custom.text.TextCleaner#1
Traceback (most recent call last):
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlpipeline.py", line 635, in _produce_block
    block_outputs = block.produce(**produce_args)
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlblock.py", line 320, in produce
    return getattr(self.instance, self.produce_method)(**produce_kwargs)
  File "/home/xals/.virtualenvs/MLBlocks.

2020-06-09 18:27:45,480 - WARNING - session - Too many errors: 1. Removing tunable sklearn.ensemble.AdaBoostClassifier
2020-06-09 18:27:45,480 - INFO - session - Creating Tunable instance from dict.
2020-06-09 18:27:45,480 - INFO - session - Obtaining default configuration for sklearn.ensemble.BaggingClassifier
2020-06-09 18:27:45,565 - ERROR - mlpipeline - Exception caught fitting MLBlock sklearn.ensemble.BaggingClassifier#1
Traceback (most recent call last):
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlpipeline.py", line 619, in _fit_block
    block.fit(**fit_args)
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlblock.py", line 300, in fit
    getattr(self.instance, self.fit_method)(**fit_kwargs)
  File "/home/xals/.virtualenvs/MLBlocks.clean/lib/python3.6/site-packages/sklearn/ensemble/bagging.py", line 244, in fit
    return self._fit(X, y, self.max_samples, sample_weight=sample_weight)
  File "/home/xals/.virtualenvs/MLBlocks.clean/lib/python3.6/site-package

{'id': 'ef66788edb4fa1b7d8403e92bf328a1d',
 'name': 'single_table.classification',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 0,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'lowercase'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'binary'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'max_features'): 1000,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.0,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8639171383183359}

During this loop, the BTBSession will build pipelines based on our templates and evaluate them
using our scoring function.

## 6. Evaluate results

When the session funishes running it will return a the best proposal available and the
obtained score.

These results are also available as the `best_proposal` attribute from the btb session object.

In [11]:
session.best_proposal

{'id': 'ef66788edb4fa1b7d8403e92bf328a1d',
 'name': 'single_table.classification',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 0,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'lowercase'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'binary'): True,
  ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
   'max_features'): 1000,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.0,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8639171383183359}

## Continue Running

If we feel that the score can still be improved and want to keep searching, we can simply run the session again which will continue tuning over the previous results.

In [12]:
session.run(10)

2020-06-09 18:27:45,624 - INFO - session - Creating Tunable instance from dict.
2020-06-09 18:27:45,625 - INFO - session - Obtaining default configuration for sklearn.ensemble.ExtraTreesClassifier
2020-06-09 18:27:45,695 - ERROR - mlpipeline - Exception caught fitting MLBlock sklearn.ensemble.ExtraTreesClassifier#1
Traceback (most recent call last):
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlpipeline.py", line 619, in _fit_block
    block.fit(**fit_args)
  File "/home/xals/Projects/MIT/MLBlocks.clean/mlblocks/mlblock.py", line 300, in fit
    getattr(self.instance, self.fit_method)(**fit_kwargs)
  File "/home/xals/.virtualenvs/MLBlocks.clean/lib/python3.6/site-packages/sklearn/ensemble/forest.py", line 250, in fit
    X = check_array(X, accept_sparse="csc", dtype=DTYPE)
  File "/home/xals/.virtualenvs/MLBlocks.clean/lib/python3.6/site-packages/sklearn/utils/validation.py", line 527, in check_array
    array = np.asarray(array, dtype=dtype, order=order)
  File "/home/xals

2020-06-09 18:28:16,093 - INFO - session - Generating new proposal configuration for single_table.classification
2020-06-09 18:28:22,502 - INFO - session - Generating new proposal configuration for single_table.classification.xgb
2020-06-09 18:28:33,460 - INFO - session - Generating new proposal configuration for single_table.classification
2020-06-09 18:28:41,217 - INFO - session - Generating new proposal configuration for single_table.classification
2020-06-09 18:28:47,272 - INFO - session - Generating new proposal configuration for single_table.classification.xgb
2020-06-09 18:28:54,772 - INFO - session - Generating new proposal configuration for single_table.classification
2020-06-09 18:29:09,485 - INFO - session - Generating new proposal configuration for single_table.classification.xgb
2020-06-09 18:29:15,094 - INFO - session - Generating new proposal configuration for single_table.classification
2020-06-09 18:29:22,733 - INFO - session - Generating new proposal configuration for

{'id': '742456ff8b6f5aa926bbefb9fab42063',
 'name': 'single_table.classification.xgb',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 11,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.3695052906894838,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.006904783754312138,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8725777466046928}

**NOTE**: If you look at the logs you will notice how the BTBSession captures the errors that finds
while executing the pipelines and automatically discards the failing tempaltes to be able to continue
the tuning session without wasting time on them.

The number of errors that we want to wait before discarding a template can be changed passing the
`max_errors` argument to the `BTBSession` when it is build.

Isn't it cool?

## Build the best pipeline

Once we are satisfied with the results, we can then build an instance of the best pipeline
by reading the `best_proposal` attribute from the `session`.

In [13]:
best_proposal = session.best_proposal
best_proposal

{'id': '742456ff8b6f5aa926bbefb9fab42063',
 'name': 'single_table.classification.xgb',
 'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
   'max_labels'): 11,
  ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
  ('xgboost.XGBClassifier#1', 'max_depth'): 3,
  ('xgboost.XGBClassifier#1', 'learning_rate'): 0.3695052906894838,
  ('xgboost.XGBClassifier#1', 'gamma'): 0.006904783754312138,
  ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
 'score': 0.8725777466046928}

In [14]:
template = templates_dict[best_proposal['name']]

pipeline = MLPipeline(template.to_dict())
pipeline.set_hyperparameters(best_proposal['config'])

pipeline.fit(dataset.data, dataset.target)

## Explore other results

Optionally, if we are interested in exploring the results of the previous proposals we can access them
in the `trials` attribute of the `session` object.

In [15]:
list(session.proposals.values())[0:2]

[{'id': 'ef66788edb4fa1b7d8403e92bf328a1d',
  'name': 'single_table.classification',
  'config': {('mlprimitives.custom.feature_extraction.CategoricalEncoder#1',
    'max_labels'): 0,
   ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
    'lowercase'): True,
   ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
    'binary'): True,
   ('mlprimitives.custom.feature_extraction.StringVectorizer#1',
    'max_features'): 1000,
   ('sklearn.impute.SimpleImputer#1', 'strategy'): 'mean',
   ('xgboost.XGBClassifier#1', 'max_depth'): 3,
   ('xgboost.XGBClassifier#1', 'learning_rate'): 0.1,
   ('xgboost.XGBClassifier#1', 'gamma'): 0.0,
   ('xgboost.XGBClassifier#1', 'min_child_weight'): 1},
  'score': 0.8639171383183359},
 {'id': 'adbd189a819483ddc869ceb94513b369',
  'name': 'single_table.classification.text',
  'config': {('mlprimitives.custom.text.TextCleaner#1', 'lower'): True,
   ('mlprimitives.custom.text.TextCleaner#1', 'accents'): True,
   ('mlprimitives.custom.tex